In [1]:
import numpy as np
import CostFuncs as cf

class Example1(cf.SecondOrderCostFunc):
    def f(self,x):
        self.num_f_calls += 1
        return x[0]**2 + x[1]**2 - x[0]*x[1] + 2.0*x[0] - 4.0*x[1] + 10.0
    
    def df(self,x):
        self.num_df_calls +=1
        return np.array([2.0*x[0]- x[1] + 2, 2.0*x[1] - x[0] - 4])
    
    def d2f(self,x):
        self.num_d2f_calls +=1
        return np.array([[2.0, -1.0],[-1, 2.0]])
    

# Simple Gradient Descent Approach
## In-class example
The in-class example was $$x^2 + y^2 - xy +2x -4y +10$$.  We had started to do gradient descent.  Let's see if we can finish that example. The cost function class is above (`Example1`)

In [3]:
def gradDescent(my_f : cf.DerivCostFunc, x_init : np.array, max_iters=10) -> (np.array, float):
    curr_x = x_init
    curr_iter=0
    best_alpha = 8.0 / 1.5
    while curr_iter < max_iters:
        curr_cost = my_f.f(curr_x)
        curr_d = my_f.df(curr_x)
        print('x',str(curr_iter),' is\t\t',curr_x,'\tcost is\t',str(curr_cost),'\tgradient is\t',curr_d,sep='')
        #Search in the steepest descent direction
        curr_alpha = best_alpha * 1.5
        best_alpha = curr_alpha
        direction = -curr_d / np.linalg.norm(curr_d)
        best_cost = my_f.f(curr_x + curr_alpha*direction)
        test_cost = best_cost
        while best_cost == test_cost:
            curr_alpha = curr_alpha / 2.0
            test_cost = my_f.f(curr_x + curr_alpha * direction)
            #print('  alpha: ',curr_alpha,'\ttest_cost:\t',test_cost)
            if test_cost < best_cost:
                best_cost = test_cost
                best_alpha = curr_alpha
                #print('Setting best_alpha to ',best_alpha)
        curr_x += direction * best_alpha
        curr_iter +=1
    print('Final result is ',curr_x,' with cost of ',my_f.f(curr_x),' and gradient of ',my_f.df(curr_x))
    print('I called f()',my_f.num_f_calls,'times and df()',my_f.num_df_calls,'times')
    return (curr_x, my_f.f(curr_x))

gradDescent(Example1(), np.array([0.0, 0.0]))

x0 is		[0. 0.]	cost is	10.0	gradient is	[ 2. -4.]
x1 is		[-0.89442719  1.78885438]	cost is	6.655728090000841	gradient is	[-1.57770876  0.47213595]
x2 is		[-0.17591016  1.57383515]	cost is	6.137594137068957	gradient is	[ 0.07434454 -0.67641954]
x3 is		[-0.20663701  1.85340164]	cost is	6.033897284961888	gradient is	[-0.26667565 -0.08655971]
x4 is		[-0.00600397  1.91852471]	cost is	6.006185094829956	gradient is	[ 0.06946734 -0.1569466 ]
x5 is		[-0.03801978  1.99085759]	cost is	6.001181495076336	gradient is	[-0.06689715  0.01973496]
x6 is		[-0.00956888  1.98246445]	cost is	6.000231263523917	gradient is	[-0.00160221 -0.02550223]
x7 is		[-0.0088714   1.99356621]	cost is	6.000063018643402	gradient is	[-0.01130901 -0.00399617]
x8 is		[-1.00531223e-03  1.99634579e+00]	cost is	6.000010690292098	gradient is	[ 0.00164359 -0.00630311]
x9 is		[-1.79470539e-03  1.99937309e+00]	cost is	6.000002488864284	gradient is	[-0.0029625   0.00054088]
Final result is  [-6.40585339e-04  1.99916237e+00]  with cost

(array([-6.40585339e-04,  1.99916237e+00]), 6.000000575396378)

## Let me also try the "default" cost parameter 
This lets me test my gradient descent and test the default cost function as well.

In [4]:
default = cf.DerivCostFunc()
gradDescent(default, np.random.randn(1,2)*5)

x0 is		[[-9.2521543   6.39003063]]	cost is	11.244325264075346	gradient is	[[-0.82282877  0.56828938]]
x1 is		[[-2.66952416  1.84371559]]	cost is	3.2443252640753464	gradient is	[[-0.82282877  0.56828938]]
x2 is		[[-0.20103786  0.13884745]]	cost is	0.24432526407534622	gradient is	[[-0.82282877  0.56828938]]
x3 is		[[ 0.03038273 -0.02098394]]	cost is	0.0369247359246538	gradient is	[[ 0.82282877 -0.56828938]]
x4 is		[[ 0.00868705 -0.00599974]]	cost is	0.010557548424653798	gradient is	[[ 0.82282877 -0.56828938]]
x5 is		[[ 0.00055117 -0.00038067]]	cost is	0.000669853112153798	gradient is	[[ 0.82282877 -0.56828938]]
x6 is		[[ 0.00016981 -0.00011728]]	cost is	0.00020636739438036045	gradient is	[[ 0.82282877 -0.56828938]]
x7 is		[[ 2.67915106e-05 -1.85036444e-05]]	cost is	3.256025021532139e-05	gradient is	[[ 0.82282877 -0.56828938]]
x8 is		[[-2.35241113e-08  1.62470045e-08]]	cost is	2.8589315623441524e-08	gradient is	[[-0.82282877  0.56828938]]
x9 is		[[-3.88419337e-09  2.68263086e-09]]	cost is

(array([[-2.01708751e-10,  1.39310809e-10]]), 2.4514061636365836e-10)

# Newton Step ... class example

In [5]:
def newtonOpt(my_f : cf.SecondOrderCostFunc, init_x : np.array, max_iters = 10) -> (np.array, float):
    curr_x = init_x
    print('curr_x is ',curr_x)
    for i in range(max_iters):
        curr_deriv = my_f.df(curr_x)
        curr_hess = my_f.d2f(curr_x)
        step = np.linalg.solve(curr_hess, curr_deriv)
        curr_x -= step
        print('curr_x is ',curr_x,' with a cost of ',my_f.f(curr_x))
    print('I called f() ',my_f.num_f_calls,' times')
    return curr_x, my_f.f(curr_x)
        
last_x, cost = newtonOpt(Example1(), np.array([0.0,0.0]))


curr_x is  [0. 0.]
curr_x is  [0. 2.]  with a cost of  6.0
curr_x is  [0. 2.]  with a cost of  6.0
curr_x is  [0. 2.]  with a cost of  6.0
curr_x is  [0. 2.]  with a cost of  6.0
curr_x is  [0. 2.]  with a cost of  6.0
curr_x is  [0. 2.]  with a cost of  6.0
curr_x is  [0. 2.]  with a cost of  6.0
curr_x is  [0. 2.]  with a cost of  6.0
curr_x is  [0. 2.]  with a cost of  6.0
curr_x is  [0. 2.]  with a cost of  6.0
I called f()  10  times


## BFGS
First, I will need to come up with a cost function that does not converge in one step...  

In [ ]:
class GPF(cf.DerivCostFunc): #Goldstein-Price Function from Wikipedia
    def f(self,x):
        self.num_f_calls += 1
        subterm11 = (19-14*x[0]+3*x[0]*x[0] - 14*x[1] + 6*x[0]*x[1] + 3 * x[1]*x[1])
        subterm21 = (18-32*x[0]+12*x[0]*x[0]+48*x[1]-36*x[0]*x[1]+27*x[1]*x[1])
        term1 = 1 + ((x[0] + x[1] + 1)**2)*subterm11
        term2 = 30 + ((2*x[0] - 3*x[1])**2)*subterm21
        return term1 * term2
    
    def df(self,x):
        self.num_df_calls +=1
        subterm11 = (19-14*x[0]+3*x[0]*x[0] - 14*x[1] + 6*x[0]*x[1] + 3 * x[1]*x[1])
        subterm21 = (18-32*x[0]+12*x[0]*x[0]+48*x[1]-36*x[0]*x[1]+27*x[1]*x[1])
        term1 = 1 + ((x[0] + x[1] + 1)**2)*subterm11
        term2 = 30 + ((2*x[0] - 3*x[1])**2)*subterm21
        dterm1x = 2*(x[0] + x[1] + 1)*subterm11 + ((x[0]+x[1]+1)**2)*(-14+6*x[0]+6*x[1])
        dterm2x = 4*(2*x[0]-3*x[1])*subterm21 + ((2*x[0] - 3*x[1])**2)*(-32 + 24*x[0] - 36*x[1])
        dterm1y = 2*(x[0]+x[1]+1)*subterm11 + ((x[0]+x[1]+1)**2)*(-14 + 6*x[0] + 6*x[1])
        dterm2y = -6*(2*x[0]-3*x[1])*subterm21 + ((2*x[0] - 3*x[1])**2)*(48 -36*x[0] + 54*x[1])
        return np.array([dterm1x*term2 + term1*dterm2x, dterm1y*term2 + term1*dterm2y])

In [8]:
def BFGS(my_f : cf.DerivCostFunc, init_x : np.array, init_B = None) -> (np.array, float):
    size_x = init_x.shape
    if size_x[0] == 1:
        N = size_x[1]
    else:
        N = size_x[0]
    if init_B is None:
        init_B = np.eye(N)
    curr_B = init_B.copy()
    curr_H = np.linalg.inv(curr_B)
    curr_x = init_x.copy()
    c1_wolfe = .1
    c2_wolfe = .5
    next_deriv = my_f.df(curr_x)
    curr_iter = 0
    while curr_iter < 10:
        curr_cost = my_f.f(curr_x)
        curr_deriv = next_deriv
        print('curr_x is ',curr_x,'with cost of ',curr_cost)
        print('curr_H is ',curr_H, 'curr_deriv is',curr_deriv)
        direction = -np.dot(curr_H,curr_deriv)
        mag_change = np.dot(direction,curr_deriv)
        print('Direction is ',direction)
        #Do a line search here...  Use the Wolfe conditions to ensure BFGS keeps things p.s.d.
        curr_alpha = 1
        cont_iter = True
        while cont_iter:
            #Evaluate at current alpha
            next_cost = my_f.f(curr_x+direction*curr_alpha)
            #print('alpha of ',curr_alpha,'gives cost of',next_cost)
            if (next_cost-curr_cost) <= curr_alpha * mag_change * c1_wolfe: #First Wolfe condition...
                new_deriv = my_f.df(curr_x+direction*curr_alpha)
                if np.dot(new_deriv,direction) >= (c2_wolfe * mag_change):
                    cont_iter = False
                    curr_alpha *= 2
            curr_alpha *= 0.5
        #print('Terminated loop with alpha of ',curr_alpha)
        movement = direction * curr_alpha #s_k in book (6.5)
        next_x = curr_x + movement
        next_deriv = my_f.df(next_x)
        diff_deriv = next_deriv-curr_deriv #y_k in book, (6.5)
        rho = 1/np.dot(diff_deriv, movement)
        mult = (np.eye(N) - rho * np.outer(movement, diff_deriv))
        #print('To do BFGS update, mult is ',mult,' movement is',movement,'rho is ',rho)
        next_H = np.dot(np.dot(mult,curr_H),mult.T) + rho * np.outer(movement,movement)
        #get ready for next iteration
        curr_deriv = next_deriv
        curr_x = next_x
        curr_H = next_H
        curr_iter += 1
    print ('curr_x of ',curr_x,'cost of ',my_f.f(curr_x))
    return curr_x, my_f.f(curr_x)


In [9]:
sol,cost = BFGS (Example1(), np.array([0.0,0.0]))

curr_x is  [0. 0.] with cost of  10.0
curr_H is  [[1. 0.]
 [0. 1.]] curr_deriv is [ 2. -4.]
Direction is  [-2.  4.]
curr_x is  [-1.  2.] with cost of  7.0
curr_H is  [[0.70918367 0.36734694]
 [0.36734694 0.69387755]] curr_deriv is [-2.  1.]
Direction is  [1.05102041 0.04081633]
curr_x is  [0.05102041 2.04081633] with cost of  6.002186588921282
curr_H is  [[0.67231962 0.3453533 ]
 [0.3453533  0.6922249 ]] curr_deriv is [0.06122449 0.03061224]
Direction is  [-0.05173447 -0.04233464]
curr_x is  [-7.14057203e-04  1.99848169e+00] with cost of  6.000001730985319
curr_H is  [[0.66801875 0.33082357]
 [0.33082357 0.67132534]] curr_deriv is [ 9.01966993e-05 -2.32256501e-03]
Direction is  [0.00070811 0.00152936]
curr_x is  [-5.95104181e-06  2.00001105e+00] with cost of  6.000000000223177
curr_H is  [[0.6715571  0.33356873]
 [0.33356873 0.666678  ]] curr_deriv is [-2.29485337e-05  2.80439420e-05]
Direction is  [ 6.05666864e-06 -1.10413658e-05]
curr_x is  [1.05626829e-07 2.00000001e+00] with cost o

KeyboardInterrupt: 